In [22]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Step 1: Load the Dataset

In [26]:
 
# Load dataset
df = pd.read_excel(r"C:\Users\SMART TECH\Desktop\AppliedNLPMaterial-master\Online Retail.xlsx")
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


# Step 2: Preprocessing the Data

In [30]:
print(df.head())

  InvoiceNo StockCode                                lower  \
0    536365    85123A   white hanging heart t-light holder   
1    536365     71053                  white metal lantern   
2    536365    84406B       cream cupid hearts coat hanger   
3    536365    84029G  knitted union flag hot water bottle   
4    536365    84029E       red woolly hottie white heart.   

                           Description  Quantity         InvoiceDate  \
0   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00   
1                  WHITE METAL LANTERN         6 2010-12-01 08:26:00   
2       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00   
3  KNITTED UNION FLAG HOT WATER BOTTLE         6 2010-12-01 08:26:00   
4       RED WOOLLY HOTTIE WHITE HEART.         6 2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  
0       2.55     17850.0  United Kingdom  
1       3.39     17850.0  United Kingdom  
2       2.75     17850.0  United Kingdom  
3       3.39     17850

In [32]:
(df.isnull().sum())

InvoiceNo           0
StockCode           0
lower          540093
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [34]:
# Drop rows with missing invoice numbers 
df.dropna(subset=['InvoiceNo'], inplace=True) # specifies that only the InvoiceNo column should be checked for missing values

# Remove leading and trailing spaces
df['Description'] = df['Description'].str.strip() # removes unwanted spaces from the Description column.

In [36]:
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Key Components:
# df['InvoiceNo'].astype(str): This converts the InvoiceNo column to string format. This step ensures that the following string operations can be performed without errors.
# .str.startswith('C'): This checks if the string in each InvoiceNo starts with 'C' and returns a boolean Series (True for those that do, False for those that don't).
# ~: This is a logical NOT operator. It negates the boolean Series, so rows where InvoiceNo does not start with 'C' are selected.
# df[...]: This filters the DataFrame, keeping only the rows where the condition is True.

In [38]:
# Display dataset after cleaning
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

# Step 3: Create Transaction Baskets

In [42]:
countries = ['France', 'United Kingdom', 'Portugal', 'Sweden']
baskets = {}

for country in countries:
    # Filter data for the given country
    country_data = df[df['Country'] == country]
    
    # Group by 'InvoiceNo' and 'Description' and sum 'Quantity'
    basket = country_data.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)
    
    # Convert quantities to binary format (1 = present, 0 = absent)
    basket = basket.map(lambda x: 1 if x > 0 else 0)
    
    # Store the basket for this country
    baskets[country] = basket

# Display a sample basket for France
print(baskets['France'].head())

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER  \
InvoiceNo                                                         
536370                                    0                   0   
536852                                    0                   0   
536974                                    0                   0   
537065                                    0                   0   
537463                                    0                   0   

Description   I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY  \
InvoiceNo                                                             
536370                                  0                         0   
536852                                  0                         0   
536974                                  0                         0   
537065                                  0                         0   
537463                                  0                         0   

Description   SET 2 TEA TOWELS I

# Step 4: Apply the Apriori Algorithm

In [46]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = {}

for country, basket in baskets.items():

    # Convert the basket to boolean values (True/False)
    basket_bool = basket.astype(bool)
    
    # Apply Apriori algorithm to find frequent itemsets
    frequent_itemsets[country] = apriori(basket_bool, min_support=0.05, use_colnames=True)

# Display frequent itemsets for France
print(frequent_itemsets['France'].head())


    support                       itemsets
0  0.071429  (4 TRADITIONAL SPINNING TOPS)
1  0.096939   (ALARM CLOCK BAKELIKE GREEN)
2  0.102041    (ALARM CLOCK BAKELIKE PINK)
3  0.094388    (ALARM CLOCK BAKELIKE RED )
4  0.068878   (ASSORTED COLOUR MINI CASES)


# Step 5: Generate Association Rules

In [49]:
from mlxtend.frequent_patterns import association_rules

rules = {}

for country, itemsets in frequent_itemsets.items():
    # Generate association rules with a minimum lift threshold of 1
    country_rules = association_rules(itemsets, metric="lift", min_threshold=1)
    
    # Filter rules involving 'Cutlery Set'
    cutlery_rules = country_rules[country_rules['antecedents'].apply(lambda x: 'Cutlery Set' in x) | 
                                   country_rules['consequents'].apply(lambda x: 'Cutlery Set' in x)]
    
    rules[country] = cutlery_rules

# Display rules for France
print(rules['France'].head())


Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, representativity, leverage, conviction, zhangs_metric, jaccard, certainty, kulczynski]
Index: []


# Step 6: Sort Rules by Confidence and Lift

In [51]:
sorted_rules = {}

for country, country_rules in rules.items():
    sorted_rules[country] = country_rules.sort_values(by=['confidence', 'lift'], ascending=False)

# Display sorted rules for France
print(sorted_rules['France'].head())


Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, representativity, leverage, conviction, zhangs_metric, jaccard, certainty, kulczynski]
Index: []


# Step 7: Extract and Analyze Top Rules

In [53]:
top_rules = {}

for country, country_rules in sorted_rules.items():
    top_rules[country] = country_rules[['antecedents', 'consequents', 'confidence', 'lift']].head()

# Display top rules for Sweden
print(top_rules['Sweden'])


Empty DataFrame
Columns: [antecedents, consequents, confidence, lift]
Index: []


# Step 8: Interpretation of Findings

In [55]:
for country, rules_df in top_rules.items():
    print(f"\nTop rules for {country}:")
    for _, row in rules_df.iterrows():
        print(f"People who buy {row['antecedents']} often also buy {row['consequents']} "
              f"with confidence {row['confidence']:.2f} and lift {row['lift']:.2f}.")



Top rules for France:

Top rules for United Kingdom:

Top rules for Portugal:

Top rules for Sweden:
